In [ ]:
import pandas as pd
from urllib.request import urlopen
import json   # 해외지수는 json 형태로 표출됨

In [ ]:
symbol = 'NII@NI225'
page = 1

In [ ]:
url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol='+symbol+'&fdtc=0&page='+str(page)
raw = urlopen(url)
data = json.load(raw)

In [ ]:
data[0]

In [ ]:
data[0]['symb']

In [ ]:
data[0]['xymd']

In [ ]:
data[0]['clos']

In [ ]:
len(data)

In [ ]:
d = dict()
for n in range(len(data)):
    date = pd.to_datetime(data[n]['xymd']).date()
    price = float(data[n]['clos'])
    d[date] = price
print(d)

In [ ]:
pd.Series(d)

In [ ]:
def read_json(d, symbol, page=1):
    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol='+symbol+'&fdtc=0&page='+str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    for n in range(len(data)):
        date = pd.to_datetime(data[n]['xymd']).date()
        price = float(data[n]['clos'])
        d[date] = price
        
    if len(data) == 10 and page<3:
        page += 1
        read_json(d, symbol, page)
        
    return (d)

In [ ]:
historical_index = pd.Series()
historical_index = read_json(historical_index, symbol, page)

In [ ]:
indices = ['SPI@SPX', 'NAS@NDX', 'NII@NI225', 'SHS@000001']

In [ ]:
indices = {
    #'SHS@000001' : '상해종합',
    'SPI@SPX' : 'S&P 500', 
    #'NAS@NDX' : 'Nasdaq 100', 
    'NII@NI225' : 'Nikkei 225'
}

In [ ]:
historical_indices = dict()
for key, value in indices.items(): 
    print (key, value)
    s = dict()
    s = read_json(s, key, 1)
    historical_indices[value] = s    
prices_df = pd.DataFrame(historical_indices)

In [ ]:
prices_df.tail()

In [ ]:
def date_format(d=''):
    if d != '':
        this_date = pd.to_datetime(d).date()
    else:
        this_date = pd.Timestamp.today().date()   # 오늘 날짜를 지정
    return (this_date)

In [ ]:
def index_global(d, symbol, start_date='', end_date='', page=1):

    end_date = date_format(end_date)
    if start_date == '':
        start_date = end_date - pd.DateOffset(months=1)
    start_date = date_format(start_date)
        
    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol='+symbol+'&fdtc=0&page='+str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    if len(data) > 0:
        
        for n in range(len(data)):
            date = pd.to_datetime(data[n]['xymd']).date()

            if date <= end_date and date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                price = float(data[n]['clos'])
                # 딕셔너리에 저장
                d[date] = price
            elif date < start_date:   
            # start_date 이전이면 함수 종료
                return (d)              

        if len(data) == 10:
            page += 1
            index_global(d, symbol, start_date, end_date, page)
        
    return (d)

In [ ]:
historical_indices = dict()
start_date = '2018-11-01'
end_date = '2018-12-31'
for key, value in indices.items(): 
    s = dict()
    s = index_global(s, key, start_date)
    historical_indices[value] = s    
prices_df = pd.DataFrame(historical_indices)

In [ ]:
prices_df

In [ ]:
historical_indices = dict()
for key, value in indices.items(): 
    s = dict()
    s = index_global(s, key)
    historical_indices[value] = s    
prices_df = pd.DataFrame(historical_indices)

In [ ]:
prices_df